# DWH Project 3 - AWS Setup
This notebook will help
- create your redshift cluster
- create appropriate roles and attach to the cluster
- create tables and load the data
- run some data validations and analysis queries
- clean up all the aws resources that were created in this process

In [ ]:
import pandas as pd
import boto3
import json

## STEP 0: Make sure you have an AWS secret and access key
	• Create a new IAM user in your AWS account
	• Give it AdministratorAccess, From Attach existing policies directly Tab
	• Take note of the access key and secret
	• Edit the file dwh.cfg in the same folder as this notebook and fill


# Load DWH Params from a file

In [ ]:
import configparser
config = configparser.ConfigParser()
config.read('dwh.cfg')

KEY = config['AWS']['KEY']
SECRET = config['AWS']['SECRET']

DWH_CLUSTER_TYPE = config['DWH']['DWH_CLUSTER_TYPE']
DWH_NUM_NODES = config['DWH']['DWH_NUM_NODES']
DWH_NODE_TYPE = config['DWH']['DWH_NODE_TYPE']
DWH_IAM_ROLE_NAME = config['DWH']['DWH_IAM_ROLE_NAME']
DWH_CLUSTER_IDENTIFIER = config['DWH']['DWH_CLUSTER_IDENTIFIER']
DWH_DB = config['DWH']['DWH_DB']
DWH_DB_USER = config['DWH']['DWH_DB_USER']
DWH_DB_PASSWORD = config['DWH']['DWH_DB_PASSWORD']
DWH_PORT = config['DWH']['DWH_PORT']

HOST = config['CLUSTER']['HOST']
DB_NAME = config['CLUSTER']['DB_NAME']
DB_USER = config['CLUSTER']['DB_USER']
DB_PASSWORD = config['CLUSTER']['DB_PASSWORD']
DB_PORT = config['CLUSTER']['DB_PORT']

ARN = config['IAM_ROLE']['ARN']

LOG_DATA = config['S3']['LOG_DATA']
LOG_JSONPATH = config['S3']['LOG_JSONPATH']
SONG_DATA = config['S3']['SONG_DATA']
BUCKET_NAME = config['S3']['BUCKET_NAME']
(DWH_DB_USER, DWH_DB_PASSWORD, DWH_DB)

pd.DataFrame({"Param":
                  ["DWH_CLUSTER_TYPE", "DWH_NUM_NODES", "DWH_NODE_TYPE", "DWH_CLUSTER_IDENTIFIER", "DWH_DB", "DWH_DB_USER", "DWH_DB_PASSWORD", "DWH_PORT", "DWH_IAM_ROLE_NAME"],
              "Value":
                  [DWH_CLUSTER_TYPE, DWH_NUM_NODES, DWH_NODE_TYPE, DWH_CLUSTER_IDENTIFIER, DWH_DB, DWH_DB_USER, DWH_DB_PASSWORD, DWH_PORT, DWH_IAM_ROLE_NAME]
             })

# Create clients for IAM, EC2, S3 and Redshift
Note: We are creating these resources in the the us-west-2 region. Choose the same region in the your AWS web console to the see these resources.

In [ ]:
import boto3

ec2 = boto3.resource('ec2',
                       region_name="us-west-2",
                       aws_access_key_id=KEY,
                       aws_secret_access_key=SECRET
                    )

s3 = boto3.resource('s3',
                       region_name="us-west-2",
                       aws_access_key_id=KEY,
                       aws_secret_access_key=SECRET
                   )

iam = boto3.client('iam',aws_access_key_id=KEY,
                     aws_secret_access_key=SECRET,
                     region_name='us-west-2'
                  )

redshift = boto3.client('redshift',
                       region_name="us-west-2",
                       aws_access_key_id=KEY,
                       aws_secret_access_key=SECRET
                       )

# Check out the sample data sources on S3

In [ ]:
dataBucket =  s3.Bucket(BUCKET_NAME)

#log_data 
for obj in dataBucket.objects.filter(Prefix="log_data"):
    print(obj)
    
#song_data 
for obj in dataBucket.objects.filter(Prefix="song_data"):
    print(obj)
    
# to read log_json file
for obj in dataBucket.objects.filter(Prefix='log_json'):
    print(obj)
    file_content = obj.get()['Body'].read().decode('utf-8')
    json_content = json.loads(file_content)
    print(json_content)

        
# for obj in dataBucket.objects.all():
#     print(obj)

# Create IAM Role that makes Redshift able to access S3 bucket (ReadOnly)

In [ ]:
from botocore.exceptions import ClientError

#1.1 Create the role, 
try:
    print("1.1 Creating a new IAM Role") 
    dwhRole = iam.create_role(
        Path='/',
        RoleName=DWH_IAM_ROLE_NAME,
        Description = "Allows Redshift clusters to call AWS services on your behalf.",
        AssumeRolePolicyDocument=json.dumps(
            {'Statement': [{'Action': 'sts:AssumeRole',
               'Effect': 'Allow',
               'Principal': {'Service': 'redshift.amazonaws.com'}}],
             'Version': '2012-10-17'})
    )    
except Exception as e:
    print(e)
    
    
print("1.2 Attaching Policy")

iam.attach_role_policy(RoleName=DWH_IAM_ROLE_NAME,
                       PolicyArn="arn:aws:iam::aws:policy/AmazonS3ReadOnlyAccess"
                      )['ResponseMetadata']['HTTPStatusCode']

print("1.3 Get the IAM role ARN")
roleArn = iam.get_role(RoleName=DWH_IAM_ROLE_NAME)['Role']['Arn']

print(roleArn)

# Step 2: Redshift Cluster

In [ ]:
try:
    response = redshift.create_cluster(        
        #HW
        ClusterType=DWH_CLUSTER_TYPE,
        NodeType=DWH_NODE_TYPE,
        NumberOfNodes=int(DWH_NUM_NODES),

        #Identifiers & Credentials
        DBName=DWH_DB,
        ClusterIdentifier=DWH_CLUSTER_IDENTIFIER,
        MasterUsername=DWH_DB_USER,
        MasterUserPassword=DWH_DB_PASSWORD,
        
        #Roles (for s3 access)
        IamRoles=[roleArn]  
    )
except Exception as e:
    print(e)

# 2.1 Describe the cluster to see its status
- run this block several times until the cluster status becomes `Available`

In [ ]:
def prettyRedshiftProps(props):
    pd.set_option('display.max_colwidth', -1)
    keysToShow = ["ClusterIdentifier", "NodeType", "ClusterStatus", "MasterUsername", "DBName", "Endpoint", "NumberOfNodes", 'VpcId']
    x = [(k, v) for k,v in props.items() if k in keysToShow]
    return pd.DataFrame(data=x, columns=["Key", "Value"])

myClusterProps = redshift.describe_clusters(ClusterIdentifier=DWH_CLUSTER_IDENTIFIER)['Clusters'][0]
prettyRedshiftProps(myClusterProps)

<h2> 2.2 Take note of the cluster <font color='red'> endpoint and role ARN </font> </h2>

<font color='red'>DO NOT RUN THIS unless the cluster status becomes "Available". Make sure you are checking your Amazon Redshift cluster in the **us-west-2** region. </font>


In [ ]:
DWH_ENDPOINT = myClusterProps['Endpoint']['Address']
DWH_ROLE_ARN = myClusterProps['IamRoles'][0]['IamRoleArn']
print("DWH_ENDPOINT :: ", DWH_ENDPOINT)
print("DWH_ROLE_ARN :: ", DWH_ROLE_ARN)

# STEP 3: Open an incoming  TCP port to access the cluster ednpoint

In [ ]:
try:
    vpc = ec2.Vpc(id=myClusterProps['VpcId'])
    defaultSg = list(vpc.security_groups.all())[0]
    print(defaultSg)
    defaultSg.authorize_ingress(
        GroupName=defaultSg.group_name,
        CidrIp='0.0.0.0/0',
        IpProtocol='TCP',
        FromPort=int(DWH_PORT),
        ToPort=int(DWH_PORT)
    )
except Exception as e:
    print(e)

# STEP 4: Make sure you can connect to the cluster

In [ ]:
%load_ext sql

In [ ]:
conn_string="postgresql://{}:{}@{}:{}/{}".format(DWH_DB_USER, DWH_DB_PASSWORD, DWH_ENDPOINT, DWH_PORT,DWH_DB)
print(conn_string)
%sql $conn_string

# Step 5: Run the data loads using the create_table and etl scripts

In [ ]:
# Execute create_tables.py python script
!python create_tables.py

In [ ]:
# Execute ety.py python script
!python etl.py

### Step 5a: Verify data loads in the tables

In [ ]:
%%time
%%sql
select count(1) from staging_events;

In [ ]:
%%time
%%sql
select count(1) from staging_songs;

In [ ]:
%%sql
select count(1) from songplay;

In [ ]:
%%time
%%sql
select count(1) from users;


In [ ]:
%%time
%%sql
select count(1) from songs;

In [ ]:
%%time
%%sql
select count(1) from artists;

In [ ]:
%%time
%%sql
select count(1) from time;

In [ ]:
#Paid vs Free Users in 2018
%%time
%%sql 
select u.level,t.year as year,count(distinct u.user_id) 
from songplay s join time t on s.start_time = t.start_time 
join users u on s.user_id = u.user_id
where t.year = 2018  
group by u.level,t.year
limit 5;

In [ ]:
#User count by time of day
%%sql select case 
        when t.hour >=0 and t.hour <6 then 'Night'
        when t.hour >=6 and t.hour <12 then 'Morning' 
        when t.hour >=12 and t.hour <18 then 'Afternoon' 
        else 'Evening' END as time_of_day
    ,count(distinct user_id) from songplay s join time t on s.start_time = t.start_time group by 1   limit 5;

In [ ]:
#Top 5 Most Listened Artist among Female users

%%sql select a.name,u.gender, count(songplay_id)
from songplay s join users u on s.user_id = u.user_id
join artists a on s.artist_id = a.artist_id
where u.gender ='F'
group by a.name,u.gender
order by  count(songplay_id) desc limit 5;

In [ ]:
#Top 5 Most Listened Artist among Male users
%%sql
select a.name,u.gender as user_gender, count(songplay_id)
from songplay s join users u on s.user_id = u.user_id
join artists a on s.artist_id = a.artist_id
where u.gender ='M'
group by a.name,u.gender
order by  count(songplay_id) desc limit 5;

# Step 6: Clean up resources

In [ ]:
#### CAREFUL!!
#-- Uncomment & run to delete the created resources
redshift.delete_cluster( ClusterIdentifier=DWH_CLUSTER_IDENTIFIER,  SkipFinalClusterSnapshot=True)
#### CAREFUL!!

- run this block several times until the cluster really deleted

In [ ]:
myClusterProps = redshift.describe_clusters(ClusterIdentifier=DWH_CLUSTER_IDENTIFIER)['Clusters'][0]
prettyRedshiftProps(myClusterProps)

In [ ]:
#### CAREFUL!!
#-- Uncomment & run to delete the created resources
iam.detach_role_policy(RoleName=DWH_IAM_ROLE_NAME, PolicyArn="arn:aws:iam::aws:policy/AmazonS3ReadOnlyAccess")
iam.delete_role(RoleName=DWH_IAM_ROLE_NAME)
#### CAREFUL!!